## Initial Tests of DMC on 1D system

In [1]:
using Pkg

# Activate project and ensure dependencies are available (first run may take time).
Pkg.activate("/Users/ismaelchona/PersonalProjects/DMC_project")
Pkg.instantiate()

# Now bring them into scope (safe if they were loaded above)
using System1D
using Plots
using StatsBase
using KernelDensity
using Statistics
using Random
using Printf

# Helpers for snapshot -> 1D x and tau -> snapshot index
xs_from_snapshot(snap) = [R[1] for R in snap]

function snap_index_from_tau(tau::Real, dt::Real, nsnaps::Int)
    return clamp(Int(round(tau/dt)) + 1, 1, nsnaps)
end

# Notebook convenience RNG (fixed seed for reproducibility).
const NB_RNG = MersenneTwister(42)

@info "Preamble complete - if you installed packages, restart the kernel and re-run this cell."


  Activating project at `~/PersonalProjects/DMC_project`
┌ Info: Preamble complete - if you installed packages, restart the kernel and re-run this cell.
└ @ Main /Users/ismaelchona/PersonalProjects/DMC_project/Notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W1sZmlsZQ==.jl:26


# 1D harmonic oscillator example
We aim to solve the Harmonic oscillator in 1D we use atomic units $$\hbar = \omega = 1 $$
$$ \displaystyle H = -\frac{1}{2}\frac{d^2}{dx^2} + \frac{1}{2} x^2  $$
This script initialiases the simulation

In [ ]:
# System setup
# Harmonic oscillator potential and Hamiltonian.
H = Hamiltonian(1, 0.5, R -> 0.5 * sum(R .^ 2))   # N=1, D=0.5, V(R)= 0.5 * x^2 (works for vector R)

# Initialize walkers uniformly in [-1, 1].
N = 10^5
W = [Walker([2*rand(NB_RNG) - 1]) for _ in 1:N]

# DMC params: dt, nsteps, nequil, targetN, ET0, pop_gain, branch_cap, nblocks.
p = DMCParams(0.005, 1000, 0, 10^5 , 0.5, 0.1, 10, 100)

# Build the simulation (no importance sampling here).
sim = DMCSim(H, p, W, NB_RNG)

# Run simulation.
sim = System1D.run_simulation!(sim)

# Extract histories for plotting.
ET_history = sim.ET_history
population_history = sim.population_history
energy_mean_history = sim.energy_mean_history
walker_positions_history = sim.walker_positions_history


Now we can plot the results

In [ ]:
# Analysis and plotting
# Create output directory for figures.
figdir = joinpath(pwd(), "figures")
mkpath(figdir)

# Post-equilibration window and snapshot times.
nequil = 500
taus   = [0.0, 0.5, 2.0]

dt = sim.params.dt
nsnaps = length(walker_positions_history)

# Time axis for time-series plots.
t = dt .* (0:(length(ET_history)-1))   # tau axis for time-series

# Compute mean for points after nequil.
start_idx = min(nequil + 1, length(energy_mean_history))
postE = energy_mean_history[start_idx:end]

Ebar = mean(postE)
SEM = (length(postE) > 1) ? std(postE) / sqrt(length(postE)) : NaN

# Significant-figure formatting for reporting.
p10 = isfinite(SEM) && SEM>0 ? floor(Int, log10(abs(SEM))) : 0
lead = isfinite(SEM) && SEM>0 ? SEM / 10.0^p10 : 1.0
sigunc = (lead < 1.5) ? 2 : 1
decimals = isfinite(SEM) && SEM>0 ? max(0, -p10 + (sigunc - 1)) : 6

println(@sprintf("Reported energy (post-eq naive SEM, nequil=%d):  E = %.*f ± %.*f (a.u.)",
                 nequil, decimals, Ebar, decimals, SEM))

# ET vs tau.
p1 = plot(t, ET_history, lw=2, label="E_T",
          xlabel="τ (a.u.)", ylabel="E_T(τ)",
          title="Reference energy vs imaginary time", legend=:bottomright)
savefig(p1, joinpath(figdir, "ET_vs_tau.png"))
display(p1)

# Population vs tau.
p2 = plot(t, population_history, lw=2, label="N_w",
          xlabel="τ (a.u.)", ylabel="N_w(τ)",
          title="Walker population vs imaginary time", legend=:topright)
savefig(p2, joinpath(figdir, "population_vs_tau.png"))
display(p2)

# Mean energy vs tau + horizontal mean line.
p3 = plot(t, energy_mean_history, lw=2, label="⟨V⟩",
          xlabel="τ (a.u.)", ylabel="⟨V⟩(τ)",
          title="Mean energy vs imaginary time", legend=:bottomright)
hline!(p3, [Ebar], ls=:dash, lw=2, label=@sprintf("post-eq mean = %.*f ± %.*f", decimals, Ebar, decimals, SEM))
savefig(p3, joinpath(figdir, "mean_energy_vs_tau.png"))
display(p3)

# Overlay KDE PDFs at selected tau + reference Gaussian.
snap_index(τ) = clamp(Int(round(τ/dt)) + 1, 1, nsnaps)
xs(snap) = [R[1] for R in snap]

idxs = snap_index.(taus)
xs_list = [xs(walker_positions_history[i]) for i in idxs]

xmin = minimum(minimum.(xs_list))
xmax = maximum(maximum.(xs_list))

p_pdf = plot(xlabel="x (a.u.)", ylabel="ϕ(x, τ)",
             title="Instantaneous walker distribution", xlims=(xmin, xmax),
             legend=:topright)

for (xvals, τ) in zip(xs_list, taus)
    k = kde(xvals; bandwidth = 0.012)
    plot!(p_pdf, k.x, k.density, lw=2, label="τ = $(τ) a.u.")
end

xgrid = range(xmin, xmax; length=800)
gauss = @. exp(-xgrid^2 / 2) / sqrt(2π)
plot!(p_pdf, xgrid, gauss, lw=2.5, ls=:dash, label="ref: exp(-x²/2)/√(2π)")

savefig(p_pdf, joinpath(figdir, "pdf_overlay_vs_tau.png"))
display(p_pdf)
